In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam
from keras.utils.np_utils import to_categorical
import pickle
import tensorflow
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Convolution2D, MaxPooling2D
from keras.layers.pooling import GlobalMaxPooling2D
from keras.layers.convolutional import Conv2D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
%matplotlib inline
py.init_notebook_mode(connected=True)

/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/luke/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def pickle_scale_factors(images, angles, df, method):
    b1_mean = images[:,:,:,0].mean()
    b2_mean = images[:,:,:,1].mean()
    ang_mean = np.nanmean(angles)

    b1_std = images[:,:,:,0].std()
    b2_std = images[:,:,:,1].std()
    ang_std = np.nanstd(angles)
    
    scale_dict = {'b1_mean': b1_mean, 'b2_mean': b2_mean, 'ang_mean': ang_mean,
                  'b1_std': b1_std, 'b2_std': b2_std, 'ang_std': ang_std}
    if method != None:
        b3_mean = images[:,:,:,2].mean()
        b3_std = images[:,:,:,2].std()
        scale_dict.update({'b3_mean': b3_mean, 'b3_std': b3_std})

    pickle_out = open("dump/scale_dict.pickle","wb")
    pickle.dump(scale_dict, pickle_out)
    pickle_out.close()

In [3]:
def scale_both(images, angles, df, method):
    pickle_in = open("dump/scale_dict.pickle","rb")
    sd = pickle.load(pickle_in)
    
    images[:,:,:,0] = (images[:,:,:,0] - sd['b1_mean']) / sd['b1_std']
    images[:,:,:,1] = (images[:,:,:,1] - sd['b2_mean']) / sd['b2_std']
    if method != None:
        images[:,:,:,2] = (images[:,:,:,2] - sd['b3_mean']) / sd['b3_std']
    angles = (angles - sd['ang_mean']) / sd['ang_std']
  
    return images, angles

In [4]:
def RGB(images, method):
    if method == 'ratio':
        b3 = (images[:,:,:,0] / images[:,:,:,1]).reshape(-1, 75, 75, 1)
        images = np.insert(images, [2], b3, axis=3)
    elif method == 'mean':
        b3 = ((images[:,:,:,0] + images[:,:,:,1]) / 2).reshape(-1, 75, 75, 1)
        images = np.insert(images, [2], b3, axis=3)
    else:
        raise Exception("Must be method type of 'ratio' or 'mean'")
    return images

In [5]:
def get_images(df):
    images = df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    images = np.stack(images).squeeze()
    return images

In [6]:
def X_y(path, scale=False, create_RGB=None, rs=1):
    '''
    Input: 
        path - path to data
        split_bands - whether or not to split the band data (pixels) into their own columns
        test_size - size of test data to break off of training data
    Returns: X_train, X_test, y_train, y_test
    '''
    np.random.seed(rs)
    kind = 'train' if len(path) == 10 else 'test'
    
    try:  # attempt to load previously created arrays
        images = np.load('dump/{}_images_{}{}.dat'.format(kind, 'scaled' if scale else 'non-scaled',
                                             '_{}'.format(create_RGB) if create_RGB != None else ''))
        angles = np.load('dump/{}_angles_{}{}.dat'.format(kind, 'scaled' if scale else 'non-scaled',
                                             '_{}'.format(create_RGB) if create_RGB != None else ''))
        if kind == 'train':
            y = np.load('dump/{}_y_{}{}.dat'.format(kind, 'scaled' if scale else 'non-scaled',
                                             '_{}'.format(create_RGB) if create_RGB != None else ''))
        else:
            ids = np.load('dump/ids.dat')
    except:  # if not array is available
        print('Creating new data arrays...')
        df = pd.read_json(path)
        df = df.replace('na',np.NaN)
        
        images = get_images(df)
        angles = df.inc_angle.values
        
        if create_RGB != None:
            images = RGB(images, create_RGB)
        
        if scale:
            if kind == 'train':
                pickle_scale_factors(images, angles, df, create_RGB)
            images, angles = scale_both(images, angles, df, create_RGB)
            
        images.dump('dump/{}_images_{}{}.dat'.format(kind, 'scaled' if scale else 'non-scaled',
                                             '_{}'.format(create_RGB) if create_RGB != None else ''))
        angles.dump('dump/{}_angles_{}{}.dat'.format(kind, 'scaled' if scale else 'non-scaled',
                                             '_{}'.format(create_RGB) if create_RGB != None else ''))
        if kind == 'train':
            y = to_categorical(df.is_iceberg, num_classes=2)
            y.dump('dump/{}_y_{}{}.dat'.format(kind, 'scaled' if scale else 'non-scaled',
                                             '_{}'.format(create_RGB) if create_RGB != None else ''))
        else:
            ids = df.id.values
            ids.dump('dump/ids.dat')
    if kind == 'train':
        return images, angles, y
    else:
        return images, angles, ids

In [7]:
# train data
images, angles, y = X_y(
    'train.json', 
    scale=True, 
    create_RGB=None
)

y = np.array([[int(x) for x in row] for row in y])

images_train, images_val, angles_train, angles_val, y_train, y_val = train_test_split(
    images, 
    angles, 
    y, 
    test_size=0.2
)

In [8]:
# create subset
mask = np.random.randint(0, len(images), size=200)
images_sub = images[mask]
anlges_sub = angles[mask]
y_sub = y[mask]

images_sub_train, images_sub_val, angles_sub_train, angles_sub_val, y_sub_train, y_sub_val = train_test_split(
    images_sub, 
    anlges_sub, 
    y_sub, 
    test_size=0.2
)

In [9]:
images_train.shape

(1283, 75, 75, 2)

In [10]:
def image_3D(band, target_name):
    data = [go.Surface(z=band)]
    layout = go.Layout(title=target_name, autosize=False, width=700, height=700, 
                       margin=dict(l=65, r=50, b=65, t=90))
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [ ]:
# 3D plot of ship
image_num = 9
image_3D(images_train[image_num,:,:,0], 'iceberg' if y[image_num][1] == 1 else 'ship')

In [ ]:
image_num = 6
image_3D(images_train[image_num,:,:,0], 'iceberg' if y[image_num][1] == 1 else 'ship')

In [ ]:
# test data
test_images, test_angles, ids = X_y(
    'test.json', 
    scale=True, 
    create_RGB=None
)

In [12]:
def build_model(shape, window_size=(3,3), stride=(1, 1), pooling_size=(2,2), activation_f='softmax',
          op_f='adam', loss_f='binary_crossentropy', pad='same', dropout=0.5):
    cnn = Sequential()
    cnn.add(BatchNormalization(input_shape = shape))
    for i in range(5):
        cnn.add(Conv2D(filters=12**i, kernel_size=window_size, strides=stride, padding=pad))
        cnn.add(MaxPooling2D(pooling_size))
    cnn.add(GlobalMaxPooling2D())
    cnn.add(Dropout(dropout))
    cnn.add(Dense(8))
    cnn.add(Dense(2, activation = activation_f))
    cnn.compile(optimizer = op_f, loss = loss_f, metrics = ['accuracy'])
    # cnn.summary()
    return cnn

# First CNN

In [21]:
def eval_cnn():
    cnn = build_model(
        shape=images_train.shape[1:], 
        window_size=(3,3), 
        stride=(1, 1), 
        pooling_size=(2,2), 
        activation_f='softmax',
        op_f='adam', 
        loss_f='binary_crossentropy', 
        pad='same', 
        dropout=0.2
    )
    cnn.fit(
        images_train, 
        y_train, 
        batch_size=None, 
        shuffle=True, 
        validation_data=(images_val, y_val), 
        epochs=5
    )
    return cnn.evaluate(images_val, y_val, verbose=0)[1]

In [ ]:
eval_cnn()

Train on 1283 samples, validate on 321 samples
Epoch 1/5


In [14]:
use_subset = 1
if use_subset:
    x_t = images_sub_train
    x_v = images_sub_val
    y_t = y_sub_train
    y_v = y_sub_val
else:
    x_t = images_train
    x_v = images_val
    y_t = y_train
    y_v = y_val

params = {'shape': [x_t.shape[1:]], 'window_size': [(3,3)], 'stride': [(1,1)], 'pooling_size': [(3,3)],
          'pad': ['same'], 'activation_f': ['relu', 'sigmoid', 'softmax'], 'op_f': ['adam'],
         'epochs': [5, 10], 'validation_data': [(x_v, y_v[:,1])], 'dropout': [0.2, 0.5, 0.7],
         'loss_f': ['sparse_categorical_crossentropy']}
model = KerasClassifier(build_fn=build_model)

grid = GridSearchCV(model, param_grid=params, n_jobs=-1, scoring='accuracy')
grid.fit(x_t, y_t[:,1])

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/luke/anaconda2/envs/py3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/luke/anaconda2/e

In [16]:
print('Best: ')
print(grid.best_score_)
print('with')
print(grid.best_params_)
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 
0.6875
with
{'activation_f': 'softmax', 'dropout': 0.2, 'epochs': 10, 'loss_f': 'sparse_categorical_crossentropy', 'op_f': 'adam', 'pad': 'same', 'pooling_size': (2, 2), 'shape': (75, 75, 2), 'stride': (1, 1), 'validation_data': (array([[[[-0.03378447,  0.19361609],
         [-0.57236388,  0.34365467],
         [-0.31516763,  0.61960823],
         ...,
         [-0.8304801 ,  0.61824206],
         [-0.49566242,  1.14775817],
         [-0.2823779 ,  1.14773762]],

        [[ 0.3269961 , -0.51441877],
         [-0.18421965, -0.13592806],
         [-0.45725818,  0.55336362],
         ...,
         [-1.43890859,  0.19221058],
         [-0.4581649 ,  0.34224916],
         [-0.35049193,  0.55195811]],

        [[-0.73935942, -0.22541983],
         [-0.45725818,  0.03413296],
         [-0.42059822, -0.04950951],
         ...,
         [-0.57328347,  0.7457625 ],
         [-0.83050623,  0.74574224],
         [-0.92589114,  0.61818275]],

        ...,

        [[ 0.13031286,  0.74695045]

In [ ]:
y_train.min()

In [ ]:
# lower drop appears to be better
acc = []
pad = ['valid', 'same']
test_var = pad
for i in test_var:
    acc.append(eval_cnn(i))

In [ ]:
plt.plot(pad, acc)

In [ ]:
predictions = cnn.predict(test_images)[:,1]

In [ ]:
len(predictions[np.where(predictions > 0.5)]) / len(predictions)

In [ ]:
out = np.insert(ids.reshape(-1, 1), [1], predictions.reshape(-1, 1), axis=1)
np.savetxt('ice_out.csv', out, delimiter=',', header='id,is_iceberg', fmt=['%s','%1.15f'], comments='')

Using 10 epochs:
val_acc of 0.851 -> 0.423

# Psuedo Label

In [ ]:
def psuedo_label(images, angles, y, test_images, test_angles, predictions, tol=0.05):
    test_ind_to_add = np.where((predictions >= 1 - tol) | (predictions <= tol))
    test_y = [0 if val < 0.5 else 1 for val in predictions[test_ind_to_add]]
    test_y = to_categorical(test_y, num_classes=2)
    new_images = np.insert(test_images[test_ind_to_add], [0], images, axis=0)
    new_angles = np.insert(test_angles[test_ind_to_add], [0], angles, axis=0)
    new_y = np.insert(test_y, [0], y, axis=0)
    print('Added {} new rows of data'.format(len(test_y)))
    return new_images, new_angles, new_y

In [ ]:
new_images, new_angles, new_y = psuedo_label(
    images, 
    angles, 
    y, 
    test_images, 
    test_angles, 
    predictions, 
    tol=0.025
)
new_images_train, new_images_val, new_angles_train, new_angles_val, new_y_train, new_y_val = train_test_split(
    new_images, 
    new_angles, 
    new_y, 
    test_size=0.2
)

# Second CNN

In [ ]:
cnn2 = model(
    shape=new_images_train.shape[1:], 
    window_size=(3,3), 
    stride=(1, 1), 
    pooling_size=(2,2), 
    activation_f='softmax',
    op_f='adam', 
    loss_f='binary_crossentropy', 
    pad='same', 
    dropout=0.5
)
cnn2.fit(
    new_images_train, 
    new_y_train, 
    batch_size=100, 
    shuffle=True, 
    validation_data=(new_images_val, new_y_val), 
    epochs=5
)

In [ ]:
predictions2 = cnn2.predict(test_images)[:,1]

In [ ]:
len(predictions2[np.where(predictions2 > 0.5)]) / len(predictions2)

In [ ]:
out2 = np.insert(ids.reshape(-1, 1), [1], predictions2.reshape(-1, 1), axis=1)
np.savetxt('ice_out2.csv', out2, delimiter=',', header='id,is_iceberg', fmt=['%s','%1.15f'], comments='')

In [ ]:
ls

In [ ]:
df = pd.read_csv('stack_minmax_bestbase.csv')
df.head()

In [ ]:
x = df.is_iceberg.values

In [ ]:
len(x[np.where(x > 0.5)]) / len(x)